In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from gdmate.io import pyvista_io
from gdmate.analysis_modules import particles
from gdmate.visualization import pyvista_vis

In [ ]:
directory = r'/mnt/15c59731-2c7b-420d-8e97-048239b4d9c8/riftinversion_overflow/072022_rip_a/output_ri_rift/particles'
length = 72
interval = 0.1
start = length-20
tsteps = np.arange(int(start/interval),int(length/interval+1),50)

In [ ]:
bounds = [400e3,600e3,550e3,620e3,0,0]

meshes = pyvista_io.pv_load_clipped_meshes(directory,tsteps,bounds=bounds,
    kind='particles',processes=4)

In [ ]:
fields = ['initial crust_upper','initial crust_lower','initial mantle_lithosphere']

base_mesh = meshes[0]
conditions = [base_mesh[field]>0.5 for field in fields]

asth = ~conditions[0] & ~conditions[1] & ~conditions[2]
lith = conditions[2]

lith_points = base_mesh.points[asth]

lith_y_values = lith_points[:,1]

lith_max_indices = np.argsort(lith_y_values)[-100000:]
#print(lith_points[:,1][lith_max_indices])

lith_max_x_values = lith_points[:,0][lith_max_indices]
lith_max_x_avg = np.median(lith_max_x_values)

#print(lith_max_x_values)
print(lith_max_x_avg)


#asth_max = np.max(base_mesh.points[:,1][lith])
#asth_max_x = base_mesh.points[:,0][base_mesh.points[:,1]==asth_max]

#print(asth_max)
#print(asth_max_x)

side = base_mesh.points[:,0] <= lith_max_x_avg

left_side_upper = side & conditions[0]
left_side_lower = side & conditions[1]
left_side_mantle = side & conditions[2]

right_side_upper = ~side & conditions[0]
right_side_lower = ~side & conditions[1]
right_side_mantle = ~side & conditions[2]

values = np.zeros(left_side_upper.shape)
values[left_side_upper] = 1
values[left_side_lower] = 2
values[left_side_mantle] = 3
values[right_side_upper] = 4
values[right_side_lower] = 5
values[right_side_mantle] = 6

base_mesh['rift_side'] = values


In [ ]:
new_meshes = particles.run_scalar_forward(meshes[0],meshes[1:],field='rift_side',
    processes=8,interpolate=True,method='KDTree')

In [ ]:
final_mesh = meshes[-1]
final_mesh_new = particles.subtract_scalar(final_mesh,base_mesh,'noninitial_plastic_strain',
    'compressional_strain',interpolate=True,processes=8)

In [ ]:
fig,axs = plt.subplots(int(len(meshes)),dpi=300,figsize=(8.5,2*len(meshes)))

for k,ax in enumerate(axs):
    pyvista_vis.pv_plot_2d(meshes[k],'rift_side',bounds=bounds[0:4],ax=ax)